In [44]:
from stellargraph.data import EdgeSplitter
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.decomposition import PCA
import stellargraph as sg
import matplotlib.pyplot as plt
import networkx as nx
import common_functions as cf
import random

In [9]:
G, mapper, reverse_mapper = cf.create_mapping_from_file_path('facebook/0.edges')

edge_splitter = EdgeSplitter(G)
splitted_graph, X, y = edge_splitter.train_test_split(p=0.2, method="global")

** Sampled 503 positive and 503 negative edges. **
Number of random walks for 'Graph': 33300


In [86]:
import random
import stellargraph as sg
from collections import Counter

def top_common_neighbors(graph):
    # Get all node ids in the graph
    all_node_ids = list(graph.nodes())

    # Initialize an empty dictionary to store the results
    result = {}

    # Iterate over each node in the graph
    for node in all_node_ids:
        # Get the neighbors of the node
        neighbors = list(graph.neighbors(node))

        # Count the number of common neighbors for each neighbor
        common_neighbors_counts = Counter([neighbor for n in neighbors for neighbor in graph.neighbors(n) if neighbor != node])

        # Get the top 5 nodes with the most common neighbors
        top_5 = [node_id for node_id, count in common_neighbors_counts.most_common(5)]

        # If the node has less than 5 neighbors, fill the remaining spots with random node ids
        while len(top_5) < 5:
            random_node = random.choice(all_node_ids)
            if random_node != node and random_node not in top_5:
                top_5.append(random_node)

        # Add the result to the dictionary
        result[node] = top_5

    return result

In [90]:
neighbor_dict = top_common_neighbors(splitted_graph)

In [122]:
# TODO 2-t külön szedni
# x1_score = []
# x2_score = []

predictions = []
for (x1, x2), y_ in zip(X,y):
    if y_ == 1:
        if x1 in neighbor_dict[x2] or x2 in neighbor_dict[x1]:
            predictions.append(1)
        else:
            predictions.append(0)
    else:
        if x1 in neighbor_dict[x2] or x2 in neighbor_dict[x1]:
            predictions.append(0)
        else:
            predictions.append(1)
            
    

In [123]:
predictions = np.array(predictions)

In [127]:
# import accuracy_score
from sklearn.metrics import accuracy_score

# Calculate the accuracy of the predictions
accuracy = accuracy_score(y, predictions)

# Confusion Matrx
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
cm = confusion_matrix(y, predictions)

In [125]:
accuracy

0.1878727634194831

In [128]:
cm

array([[  5, 498],
       [319, 184]], dtype=int64)